In [1]:
import tensorflow as tf
import numpy as np
import yaml
import glob
import os
import pandas as pd

from tqdm import tqdm

2022-01-28 07:33:03.311370: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/oracle/instantclient_19_11:
2022-01-28 07:33:03.311396: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
filepaths = yaml.load(open("../config/filepaths.yaml"))

/home/guilherme-resende/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [3]:
class Segmenter():
    def __init__(self, model_path="../data/models/GloVe_w_LSTM", max_tokens=200):
        self.model = tf.keras.models.load_model(model_path)
        self.max_tokens = max_tokens

    def predict(self, X):
            preds = [0]

            X_splitted = X.split()
            n_tokens = len(X_splitted)
            i = 0
            while i < n_tokens:
                x_sliced = X_splitted[i:i + self.max_tokens]
                x_to_pred = np.array([' '.join(x_sliced)])

                partial_preds = self.model.predict(x_to_pred).numpy().reshape(1,-1)
                partial_preds = (partial_preds > 0.5).astype(int)

                punctuations = np.argwhere(partial_preds == 1)
                # Shift all index by 1
                punctuations += 1
                if punctuations.shape[0] == 0:
                    preds += [n_tokens]
                    break
                else:
                    punctuations = punctuations[:, 1].reshape(-1)
                    last_punctuation = punctuations[-1]

                # Adds only the text until the last punctuation, because
                # otherwise the next phrase will lack the entire context
                punctuations += i
                preds += punctuations.tolist()

                i += last_punctuation

            return preds
    
    def segment(self, X, punctuations):
        sentences = []
        
        X_splitted = X.split()
        for i in range(1, len(punctuations)):
            left = punctuations[i-1]
            right = punctuations[i]
            
            sentences.append(' '.join(X_splitted[left:right]))
        
        return sentences

In [4]:
segmenter = Segmenter()

2022-01-28 07:33:06.258976: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-28 07:33:06.258994: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kunumi): /proc/driver/nvidia/version does not exist
2022-01-28 07:33:06.259145: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(filepaths["preprocessed_data"], "*"))]

for filename in tqdm(filenames, total=len(filenames)):
    with open(os.path.join(filepaths["preprocessed_data"], filename)) as file:
        text = file.read()
    
    
    sentences = segmenter.segment(text, segmenter.predict(text))
    
    df = pd.DataFrame(sentences, columns=["text"])
    df.to_csv(os.path.join(filepaths["model_based_segmentated_data"], filename), index=False)

100%|███████████████████████████████████████████████████████████████| 552/552 [04:19<00:00,  2.12it/s]
